In [1]:
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval = fiqa_eval["baseline"]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
contexts = []

for row in fiqa_eval:
    contexts.append(row['contexts'][0])

contexts

['Just have the associate sign the back and then deposit it.  It\'s called a third party cheque and is perfectly legal.  I wouldn\'t be surprised if it has a longer hold period and, as always, you don\'t get the money if the cheque doesn\'t clear. Now, you may have problems if it\'s a large amount or you\'re not very well known at the bank.  In that case you can have the associate go to the bank and endorse it in front of the teller with some ID.  You don\'t even technically have to be there.  Anybody can deposit money to your account if they have the account number. He could also just deposit it in his account and write a cheque to the business."I have checked with Bank of America, and they say the ONLY way to cash (or deposit, or otherwise get access to the funds represented by a check made out to my business) is to open a business account. They tell me this is a Federal regulation, and every bank will say the same thing.  To do this, I need a state-issued ""dba"" certificate (from t

In [3]:
from functions import initialize_llm, initialize_vector_database, add_text, pass_prompt_multi_query

llm = initialize_llm()
vectorstore = initialize_vector_database()

In [4]:
vectorstore.add_texts(contexts)

['bea81916-aa83-11ee-be7e-12c3a083d23c',
 'bea819de-aa83-11ee-be7e-12c3a083d23c',
 'bea819f2-aa83-11ee-be7e-12c3a083d23c',
 'bea81a06-aa83-11ee-be7e-12c3a083d23c',
 'bea81a10-aa83-11ee-be7e-12c3a083d23c',
 'bea81a24-aa83-11ee-be7e-12c3a083d23c',
 'bea81a38-aa83-11ee-be7e-12c3a083d23c',
 'bea81a56-aa83-11ee-be7e-12c3a083d23c',
 'bea81a6a-aa83-11ee-be7e-12c3a083d23c',
 'bea81a74-aa83-11ee-be7e-12c3a083d23c',
 'bea81a88-aa83-11ee-be7e-12c3a083d23c',
 'bea81a92-aa83-11ee-be7e-12c3a083d23c',
 'bea81aa6-aa83-11ee-be7e-12c3a083d23c',
 'bea81ab0-aa83-11ee-be7e-12c3a083d23c',
 'bea81aba-aa83-11ee-be7e-12c3a083d23c',
 'bea81ace-aa83-11ee-be7e-12c3a083d23c',
 'bea81ad8-aa83-11ee-be7e-12c3a083d23c',
 'bea81aec-aa83-11ee-be7e-12c3a083d23c',
 'bea81af6-aa83-11ee-be7e-12c3a083d23c',
 'bea81b0a-aa83-11ee-be7e-12c3a083d23c',
 'bea81b14-aa83-11ee-be7e-12c3a083d23c',
 'bea81b1e-aa83-11ee-be7e-12c3a083d23c',
 'bea81b28-aa83-11ee-be7e-12c3a083d23c',
 'bea81b3c-aa83-11ee-be7e-12c3a083d23c',
 'bea81b46-aa83-

In [5]:
answers = []

for row in fiqa_eval:
    answers.append(pass_prompt_multi_query(row['question'], vectorstore))

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4955 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [6]:
answers[0]

"To deposit a cheque issued to an associate in your business into your business account, you can have the associate sign the back of the cheque and then deposit it. If the cheque is for a large amount or you're not well known at the bank, you can have the associate go to the bank and endorse it in front of the teller with some ID. Alternatively, the associate can deposit the cheque into their own account and write a cheque to the business."

In [7]:
new_dataset = []
for row in range(len(fiqa_eval)):
    new_dataset.append({
        'question': fiqa_eval[row]['question'],
        'contexts': fiqa_eval[row]['contexts'],
        'answer': answers[row],
        'ground_truths': fiqa_eval[row]['ground_truths']
    })

In [8]:
import pandas as pd
import datasets

new_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=new_dataset))

In [9]:
import pickle
with open("multi_answers.pkl", "wb") as f:
    pickle.dump(new_dataset, f)